<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em>Find Lower TTL(Time To Live) for Route53 records and change it to a higher TTL value</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Change-AWS-Route53-TTL-value"><strong><u>Change AWS Route53 TTL value</u></strong></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Get TTL under X hours</a><br>2)<a href="#2" target="_self" rel="noopener"> Change the TTL value</a></p>

In [ ]:
if hosted_zone_id and not record_name and not record_type:
    raise SystemExit("Provide a Record Name and Record Type!")
elif record_name and not hosted_zone_id and not record_type:
    raise SystemExit("Provide a Hosted Zone ID and Record Type!")
elif record_type and not hosted_zone_id and not record_name:
    raise SystemExit("Provide a Hosted Zone ID and Record Name!")
elif record_type and hosted_zone_id and not record_name:
    raise SystemExit("Provide a Record Name!")
elif record_name and hosted_zone_id and not record_type:
    raise SystemExit("Provide a Record Type!")
elif hosted_zone_id and record_name and not record_type:
    raise SystemExit("Provide a Record Type!")
elif hosted_zone_id and record_type and not record_name:
    raise SystemExit("Provide a Record Name!")

<h3 id="Get-TTLs-under-X-hours"><a id="1" target="_self" rel="nofollow"></a>Get TTLs under X hours</h3>
<p>Using unSkript's Get Route53 TTL Under Hours , we will find the hosted zones and records that have a TTL under given threshold hours. A lower TTL means more queries arrive at the name servers because the cached values expire sooner.</p>
<blockquote>
<p>This action takes the following parameters: <code>threshold(in hours)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>lower_ttl_records</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Tuple, Optional
import pprint
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_get_ttl_for_route53_records.aws_get_ttl_for_route53_records import aws_get_ttl_for_route53_records


from beartype import beartype
@beartype
def aws_get_ttl_under_given_hours_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_ttl_under_given_hours(handle, threshold: int = 1) -> Tuple:
    """aws_get_ttl_under_x_hours Returns TTL for records in a hosted zone

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type threshold: str
        :param threshold: (In hours) A threshold in hours to verify route 53 TTL is within the threshold.

        :rtype: List of details with the record type, record name and record TTL.
    """
    result = []
    try:
        route_client = handle.client('route53')
        seconds = threshold * 3600
        hosted_zones = aws_get_paginator(route_client, "list_hosted_zones", "HostedZones")
        for zone in hosted_zones:
            record_ttl_data = aws_get_ttl_for_route53_records(handle, zone['Id'])
            for record_ttl in record_ttl_data:
                if isinstance(record_ttl['record_ttl'], str):
                    continue
                elif record_ttl['record_ttl'] < seconds:
                    records = {}
                    records["hosted_zone_id"] = zone['Id']
                    records["record_name"] = record_ttl['record_name']
                    records["record_type"] = record_ttl['record_type']
                    records["record_ttl"] = record_ttl['record_ttl']
                    result.append(records)
    except Exception as e:
        pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "threshold": "int(threshold_ttl)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not hosted_zone_id and not record_name and not record_type",
    "condition_result": true
    }''')
task.configure(outputName="lower_ttl_records")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_ttl_under_given_hours, lego_printer=aws_get_ttl_under_given_hours_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Unused-Log-Streams&para;">Create List of Lower TTL records</h3>
<p>This action filters the output from Step 1 to get the non empty values</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_lower_ttl_records</code></p>
</blockquote>

In [ ]:
# print(lower_ttl_records)
all_lower_ttl_records = []
try:
    if lower_ttl_records[0] == False:
        if len(lower_ttl_records[1])!=0:
            all_lower_ttl_records=lower_ttl_records[1]
except Exception:
    data_dict = {}
    data_dict["hosted_zone_id"] = hosted_zone_id
    data_dict["record_name"] = record_name
    data_dict["record_type"] = record_type
    all_lower_ttl_records.append(data_dict)
print(all_lower_ttl_records)
task.configure(outputName="all_lower_ttl_records")

<h3 id="Change-TTL-Value"><a id="2" target="_self" rel="nofollow"></a>Change TTL Value</h3>
<p>This action changes the TTL value for a record that has a lower value to a higher one. By default<span style="color: rgb(45, 194, 107);"> 86400 </span>seconds is considered if no value is given,</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>hosted_zone_id, record_name, record_type, new_ttl</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from unskript.enums.aws_route53_record_type_enums import Route53RecordType
from typing import Dict
import pprint

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_update_ttl_for_route53_records_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_update_ttl_for_route53_records(handle, hosted_zone_id: str, record_name: str, record_type:Route53RecordType, new_ttl:int) -> Dict:
    """aws_update_ttl_for_route53_records updates the TTL for a Route53 record in a hosted zone.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type hosted_zone_id: string
        :param hosted_zone_id: ID of the hosted zone in Route53

        :type record_name: string
        :param record_name: Name of record in a hosted zone. Eg: example.com

        :type record_type: string
        :param record_type: Record Type of the record.

        :type new_ttl: int
        :param new_ttl: New TTL value for a record. Eg: 300

        :rtype: Response of updation on new TTL
    """

    route53Client = handle.client('route53')
    new_ttl_value = int(new_ttl)

    response = route53Client.change_resource_record_sets(
        HostedZoneId=hosted_zone_id,
        ChangeBatch={
            'Changes': [
                {
                    'Action': 'UPSERT',
                    'ResourceRecordSet': {
                        'Name': record_name,
                        'Type': record_type,
                        'TTL': new_ttl_value
                    }
                }
            ]
        }
    )
    return response



task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "hosted_zone_id": "\\"iter.get(\\\\\\"hosted_zone_id\\\\\\")\\"",
    "new_ttl": "int(new_ttl)",
    "record_name": "\\"iter.get(\\\\\\"record_name\\\\\\")\\"",
    "record_type": "\\"iter.get(\\\\\\"record_type\\\\\\")\\""
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_lower_ttl_records",
    "iter_parameter": ["hosted_zone_id","record_name","record_type"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_lower_ttl_records)!=0",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_update_ttl_for_route53_records, lego_printer=aws_update_ttl_for_route53_records_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able change the TTL(time to live) to a higher value. As a result, there are fewer queries received by the name servers which will help in saving your AWS costs. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>